<h2>Pdf to CSV Extraction</h2>

<p>This code is configured for SCE-2021 extraction, as pdf format is changing with every report we have to make the column changes for other year pdf reports.</p>

In [2]:
#!pip install tabula-py
import time
from tabula.io import convert_into
from urllib.parse import unquote
import pandas as pd
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

def readPDF(inputfile, page, outfile):
    times = str(round(time.time() * 1000))
    convert_into(inputfile, outfile, output_format="csv", pages=page)
    print("csv created at ",outfile)
    
def formatFile(outfile):
    data = pd.read_csv(outfile)
    data = data[data.ne(data.columns).any(1)]
    data = data.loc[:, ~data.columns.str.contains('^Unnamed')]
    data = data.loc[:, ~data.columns.str.contains('^N/A')]
    data.dropna(inplace=True)

    data = data[['County', 'Circuit', 'Reasons for De?\rEnergization','FPI\rValue','Wind\rSustained', 'Gust\rSustained','Thresholds\r(Sustained/\rGust)', 'POC\r*(Not Originally in\rScope)']]
    data = data[data.ne(data.columns).any(1)]
    data.reset_index(drop=True,inplace=True)
    data.columns = ['County', 'Circuit', 'Reason', 'FPI','WindSustained', 'GustSustained','Thresholds', 'Month']
    data['Month'] = pd.DatetimeIndex(pd.to_datetime(str(pd.datetime.now().year) 
                                                    + '/'
                                                    + data['Month'].str[:4].replace('?',''), format='%Y/%m/%d')).month_name()
    data['pspsActivated'] = 1
    #data.head()
    data.to_csv(outfile,index=False)
    

#inputfile = input("Enter pdf file name: ")
#page = input("Enter page numbers: ")

inputfile = 'https://www.cpuc.ca.gov/uploadedFiles/CPUCWebsite/Content/News_Room/NewsUpdates/2021/Jan.%2012-21,%202021%20SCE%20PSPS%20Post%20Event%20Report.pdf'
outfile = f"{unquote(inputfile.split('/')[-1])[:-4]}.csv"
page = '29-34'

readPDF(inputfile,page, outfile)
formatFile(outfile)

print("Completed")


csv created at  Jan. 12-21, 2021 SCE PSPS Post Event Report.csv
Completed
